# Gaussian Graphical Model on a 4‑Cycle
This notebook simulates a 4‑dimensional Gaussian random vector whose precision matrix corresponds to the cyclic graph 1–2–3–4–1.
We verify conditional independence structure and estimate covariance/precision matrices.

## 1. Define the graph and precision matrix

In [8]:
import numpy as np
import pandas as pd

d = 4
Theta = np.array([
    [10,  5,  0,  3],
    [ 5, 10,  5,  0],
    [ 0,  5, 10,  5],
    [ 3,  0,  5, 10]
])
Sigma = np.linalg.inv(Theta)
Sigma

array([[ 0.29411765, -0.26470588,  0.23529412, -0.20588235],
       [-0.26470588,  0.38823529, -0.31176471,  0.23529412],
       [ 0.23529412, -0.31176471,  0.38823529, -0.26470588],
       [-0.20588235,  0.23529412, -0.26470588,  0.29411765]])

## 2. Simulate Gaussian sample

In [9]:
N = 10**6
mean = np.zeros(d)
X = np.random.multivariate_normal(mean, Sigma, size=N)
X[:5]

array([[-0.61387469,  1.03123743, -0.21641909,  0.38343334],
       [-0.33425414,  0.40893535, -0.28014723,  0.60087478],
       [ 0.28666776, -0.04199979, -0.85809002,  0.07431151],
       [-0.01827194, -0.36529607,  0.20071357, -0.46359253],
       [ 0.89308581, -1.33016823,  1.07547789, -1.26731131]])

## 4. Estimate covariance and precision

In [10]:
EstimSigma = np.cov(X, rowvar=False)
EstimTheta = np.linalg.inv(EstimSigma)
EstimSigma, EstimTheta

(array([[ 0.29433154, -0.26493106,  0.23590508, -0.2065122 ],
        [-0.26493106,  0.38835123, -0.31228855,  0.23612319],
        [ 0.23590508, -0.31228855,  0.38899131, -0.26570873],
        [-0.2065122 ,  0.23612319, -0.26570873,  0.29505332]]),
 array([[10.00274606,  4.99382159, -0.01218043,  2.99368126],
        [ 4.99382159, 10.00855449,  4.99632564, -0.0149105 ],
        [-0.01218043,  4.99632564, 10.00935348,  5.0069216 ],
        [ 2.99368126, -0.0149105 ,  5.0069216 , 10.00542956]]))

## 5. Maximum-likelihood estimation

In [11]:
from scipy.optimize import minimize

def construct_symmetric_matrix(theta_vec):
    Theta_mat = np.zeros((d, d))
    idx = np.triu_indices(d)
    Theta_mat[idx] = theta_vec
    Theta_mat = Theta_mat + Theta_mat.T - np.diag(np.diag(Theta_mat))
    return Theta_mat

def log_likelihood(theta_vec, S):
    Theta_mat = construct_symmetric_matrix(theta_vec)
    det = np.linalg.det(Theta_mat)
    if det <= 0:
        return -np.inf
    return np.log(det) - np.trace(S @ Theta_mat)

A = np.random.randn(d, d)
Theta0 = A @ A.T + d * np.eye(d)
theta0_vec = Theta0[np.triu_indices(d)]

res = minimize(lambda t: -log_likelihood(t, EstimSigma),
               theta0_vec)

Theta_MLE = construct_symmetric_matrix(res.x)
Theta_MLE

C:\Users\20254817\AppData\Local\anaconda3\Lib\site-packages\scipy\optimize\_numdiff.py:596: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0
C:\Users\20254817\AppData\Local\anaconda3\Lib\site-packages\scipy\optimize\_numdiff.py:596: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0


array([[10.00312117,  4.994186  , -0.01200583,  2.99389956],
       [ 4.994186  , 10.00885584,  4.99619159, -0.01503403],
       [-0.01200583,  4.99619159, 10.008951  ,  5.0067704 ],
       [ 2.99389956, -0.01503403,  5.0067704 , 10.00562312]])